In [10]:
import copy
import easydict

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim

from skimage.exposure import exposure
from skimage.feature import hog

import numpy as np
import matplotlib.pyplot as plt


# PreParing Data

In [11]:
# Data transforms
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.255]

data_transform = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
        transforms.Grayscale(num_output_channels=3)
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
        transforms.Grayscale(num_output_channels=3)
    ])
}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [12]:
# Loading Datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transform['train'],
                                             download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=data_transform['val'], download=True)

# Defining class names
class_names = train_dataset.classes
print(f'Class names are {class_names}')

# Creating DataLoaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False, num_workers=2)
print('DataLoaders Are Ready.')

Files already downloaded and verified
Files already downloaded and verified
Class names are ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
DataLoaders Are Ready.


# HOG Feature Extraction

In [20]:
# HOG Parameters
orientations = 9 
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
block_norm = 'L2-Hys'


class HOGFeatureExtractor(nn.Module):
    def __init__(self):
        super(HOGFeatureExtractor, self).__init__()

    def forward(self, x):
        hog_features_list = []
        # Iterating over data in each batch and extract hog features 
        for image in x:
            # Convert Image to cpu tensor and numpy for hog
            # image = image.cpu()
            # np_image = image.numpy().squeeze()
            np_image = image

            # Extract HOG features from each channel 
            img_channels = []
            for channel_num in range(3):
                hog_channel = hog(
                    np_image[:, :, channel_num],
                    orientations=orientations,
                    pixels_per_cell=pixels_per_cell,
                    cells_per_block=cells_per_block,
                    block_norm=block_norm
                )
                hog_channel = exposure.rescale_intensity(hog_channel, in_range=(0, 10))
                img_channels.append(hog_channel)

            img_hog = np.dstack((np.array(img_channels[0]), np.array(img_channels[1]), np.array(img_channels[2])))
            hog_features_list.append(img_hog)

        return torch.tensor(np.array(hog_features_list), dtype=torch.float32)


In [24]:
h = HOGFeatureExtractor()
print(train_dataset[0][0])
img = h([train_dataset[0][0]])
plt.imshow(img)
plt.show()

tensor([[[-0.9599, -0.9599, -0.9599,  ...,  0.2029,  0.2029,  0.2029],
         [-0.9599, -0.9599, -0.9599,  ...,  0.2029,  0.2029,  0.2029],
         [-0.9599, -0.9599, -0.9599,  ...,  0.2029,  0.2029,  0.2029],
         ...,
         [ 0.5797,  0.5797,  0.5797,  ..., -0.3082, -0.3082, -0.3082],
         [ 0.5797,  0.5797,  0.5797,  ..., -0.3082, -0.3082, -0.3082],
         [ 0.5797,  0.5797,  0.5797,  ..., -0.3082, -0.3082, -0.3082]],

        [[-0.9599, -0.9599, -0.9599,  ...,  0.2029,  0.2029,  0.2029],
         [-0.9599, -0.9599, -0.9599,  ...,  0.2029,  0.2029,  0.2029],
         [-0.9599, -0.9599, -0.9599,  ...,  0.2029,  0.2029,  0.2029],
         ...,
         [ 0.5797,  0.5797,  0.5797,  ..., -0.3082, -0.3082, -0.3082],
         [ 0.5797,  0.5797,  0.5797,  ..., -0.3082, -0.3082, -0.3082],
         [ 0.5797,  0.5797,  0.5797,  ..., -0.3082, -0.3082, -0.3082]],

        [[-0.9599, -0.9599, -0.9599,  ...,  0.2029,  0.2029,  0.2029],
         [-0.9599, -0.9599, -0.9599,  ...,  0

ValueError: negative dimensions are not allowed

# Setup pretrained model

In [84]:
# Load pretrained model
pretrained_model = models.resnet18(pretrained=True)

# Freeze all trainable layers
for param in pretrained_model.parameters():
    param.requires_grad = False

# Modifying last classification layer for our dataset
num_features = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_features, 10)

In [85]:
# Defining loss function and optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pretrained_model.fc.parameters(), lr=0.001)

# Train Function 

In [86]:
hog_features_extractor = HOGFeatureExtractor()

acc_list = easydict.EasyDict({'train': [], 'val': []})
loss_list = easydict.EasyDict({'train': [], 'val': []})


def train_model(model, criterion, optimizer, epoch_num=25):
    # Copy the best model weights for loading at the End
    best_model_wts = copy.deepcopy(model.state_dict())
    best_accuracy = 0.0

    # Iterating over epochs
    for epoch in range(1, epoch_num + 1):
        print(f'Epoch {epoch}/{epoch_num}:')

        # Each epoch has two phase Train and Validation
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            # For calculating Loss and Accuracy at the end of epoch
            running_loss = 0.0
            running_corrects = 0.0

            # Iterating over data for training and validation
            for idx, data in enumerate(train_loader, 0):
                inputs, labels = data

                # Transfer data and labels to Cuda if is available
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Extract HOG features
                hog_features = hog_features_extractor(inputs)

                # Forward Pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(hog_features)
                    loss = criterion(outputs, labels)
                    _, predictions = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(predictions == labels.data)



In [87]:
hog_features_extractor = HOGFeatureExtractor()

acc_list = easydict.EasyDict({'train': [], 'val': []})
loss_list = easydict.EasyDict({'train': [], 'val': []})


def train_model(model, criterion, optimizer, epoch_num=25):
    # Copy the best model weights for loading at the End
    best_model_wts = copy.deepcopy(model.state_dict())
    best_accuracy = 0.0

    # Iterating over epochs
    for epoch in range(1, epoch_num + 1):
        print(f'Epoch {epoch}/{epoch_num}:')

        ### Training Phase ###
        # Set model to train mode
        model.train()

        # For calculating Loss and Accuracy at the end of epoch
        running_loss = 0.0
        running_corrects = 0.0

        # Iterating over training data
        for data in train_loader:
            inputs, labels = data

            # Transfer data and labels to Cuda if is available
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # Extract HOG features
            hog_features = hog_features_extractor(inputs)

            # Forward Pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, predictions = torch.max(outputs, 1)

            # Backpropagation and updating weights   
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(predictions == labels.data)

        train_loss = running_loss / len(train_dataset)
        train_acc = running_corrects.double() / len(train_dataset)

        # Show train details        
        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}')

        # Save loss and accuracy
        acc_list['train'].append(train_acc)
        loss_list['train'].append(train_loss)

        ### Validation Phase ###
        # Set model to evaluation mode
        model.eval()

        # For calculating Loss and Accuracy at the end of epoch
        running_loss = 0.0
        running_corrects = 0.0

        # Iterating over validation data
        for data in test_loader:
            inputs, labels = data

            # Transfer data and labels to Cuda if is available
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # Extract HOG features
            # hog_features = hog_features_extractor(inputs)

            # Forward Pass
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, predictions = torch.max(outputs, 1)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(predictions == labels.data)

        val_loss = running_loss / len(test_dataset)
        val_acc = running_corrects.double() / len(test_dataset)

        # Show details
        print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}', end='\n')

        # Save loss and accuracy
        acc_list['val'].append(val_acc)
        loss_list['val'].append(val_loss)

        # Copy model weights if the weights are better 
        if val_acc > best_accuracy:
            best_accuracy = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            print('Best model weights based on Higher Val_acc updated!', end='\n')

    # Load best model weights
    print('Loading best model weights')
    model.load_state_dict(best_model_wts)
    return model

In [88]:
model = pretrained_model.to(device)

# Train model
model = train_model(model, criterion, optimizer, 100)

Epoch 1/100:
Train Loss: 1.1126, Train Accuracy: 0.6208
Validation Loss: 0.9131, Validation Accuracy: 0.6865
Best model weights based on Higher Val_acc updated!
Epoch 2/100:
Train Loss: 0.9429, Train Accuracy: 0.6731
Validation Loss: 0.8886, Validation Accuracy: 0.6884
Best model weights based on Higher Val_acc updated!
Epoch 3/100:
Train Loss: 0.9180, Train Accuracy: 0.6818
Validation Loss: 0.8917, Validation Accuracy: 0.6877
Epoch 4/100:
Train Loss: 0.9076, Train Accuracy: 0.6866
Validation Loss: 0.9079, Validation Accuracy: 0.6899
Best model weights based on Higher Val_acc updated!
Epoch 5/100:


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x00000293013CFF40>
Traceback (most recent call last):
  File "C:\Users\Mahdiar\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "C:\Users\Mahdiar\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 1442, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "C:\Users\Mahdiar\AppData\Local\Programs\Python\Python310\lib\multiprocessing\process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "C:\Users\Mahdiar\AppData\Local\Programs\Python\Python310\lib\multiprocessing\popen_spawn_win32.py", line 108, in wait
    res = _winapi.WaitForSingleObject(int(self._handle), msecs)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
plt.subplot(1, 2, 1)
plt.plot()